In [1]:
from sys import platform
import sys, os
# identification de la base de prix utilisée : directory principal
# sur ubuntu

path=''
if platform == 'linux' :
    Initialpath ='/workspace/NeuralPricing/A_GeneralizedSABR'
# sur. mac osx
if platform == 'darwin' :
    Initialpath ='/Users/olivier/keras/NeuralPricing/A_GeneralizedSABR'
import keras,tensorflow,pkg_resources
dataDirectory='Generate_SABR-C#/Data'
#datafile='Data/Baby_Learningbase1.CSV'
os.chdir(Initialpath)
# le package identifie la methode de calibration : sous-directory
ResultDirectory='Data'
#datafile='Data/Baby_Learningbase1.CSV'

/Users/olivier/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from __future__ import print_function
import tensorflow as tf
from math import sqrt, exp, log, erf,floor
import numpy 
import pandas
from decimal import *
import keras
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras import callbacks
from keras.layers import Activation
from keras.layers import Lambda
from keras.models import Sequential
from keras.models import model_from_json
import matplotlib
import matplotlib.pyplot as pyplot
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import mpl_toolkits
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d.axes3d import Axes3D
from sklearn import preprocessing
from sklearn import model_selection
import joblib
import seaborn as sns
import scipy
from IPython.display import display
import time
%matplotlib inline
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual,FloatProgress
import ipywidgets as widgets


In [3]:
getcontext().prec = 8

from util_functions_ExtendedSABR import *

In [4]:
params=metaparameters()

params.INPUT_DIM  = 12
params.INPUT_OPTION = 12
params.INPUT_VOL =13
params.NB_NEURON_PRINCIPAL =8
params.ACTIVATION_PRINCIPALE = 'tanh'
params.ACTIVATION_PRINCIPALE_FINALE = 'linear'
params.INITIAL_LEARNING_NB_EPOCH=5
params.LEARNINGBASE_ORIGIN="Learningbase1_4XNeuron_15000-11Layers"
params.LEARNINGBASE_BUT="Learningbase1_4XNeuron_15000-11Layers-Next"
params.ADDITIONAL_LEARNING_NB_EPOCH = 5
params.BATCH_SIZE_PRINCIPAL = 32768
params.OPTIMIZER='adamax'
params.NBLAYERS = 11
params.PATH = Initialpath
params.VERBOSE_FLAG=2

In [5]:
def convertVector(y):
    return np.array([pandas.to_numeric(x.replace(",",".")) for x in y])

def convertTensor(Z):
    return np.array([convertVector(z) for z in Z])

In [6]:
def Xa(i):
    return np.asarray(list(filter(lambda x:(x[12]>0 and x[11]>0),\
        convertTensor((pandas.read_csv(dataDirectory+\
            "/ResultatGeneratedprices-extendedSABR-100-0,05-10-0-0-100000-"+\
                                str(i)+".CSV", sep=";",decimal=",",header=None).values)[1:,1:]))))


In [7]:
def Xb(i):
    ax=pandas.read_csv(dataDirectory+\
            "/task"+str(i)+".csv", sep=";",decimal=",")
    ax['beta2']=0
    ax['d']=0
    ax['gamma']=1
    ax['omega']=0
    ax['lambda']=0
    ax['F0']=ax['forward']
    ax['option']=ax['prix']
    ax['alpha0']=ax['alpha']
    az=  ax[['alpha0','beta','beta2','d','gamma','nu','omega','lambda','rho',\
                'maturity','strike','option','vol']].values
    return np.asarray([pandas.to_numeric(az[j]) for j in range(0,az.shape[0])])

In [8]:
startTime = time.time()
Xtotala =np.concatenate([Xa(j) for j in range(0,98)])
Xtotalb =np.concatenate([Xb(j) for j in range(0,21)])
Xtotal = np.concatenate([Xtotala,Xtotalb])
endTime = time.time()
print('temps de fabrication du ficher',endTime - startTime)

temps de fabrication du ficher 299.4744770526886


In [9]:
Xtotal.shape

(1984919, 13)

In [10]:
from sklearn import model_selection
XTraining,XValidation = model_selection.train_test_split( Xtotal,  test_size=0.10, random_state=3)

In [11]:
XTraining.shape

(1786427, 13)

In [12]:
XValidation.shape

(198492, 13)

In [13]:
ResultPath = Initialpath + '/' + ResultDirectory
os.chdir(ResultPath)
# on sauve le resultat 
np.savetxt("LearningBaseFileB.CSV",XTraining,delimiter=",")
np.savetxt("Notation.CSV",XValidation,delimiter=",")